In [1]:
import random
from datetime import datetime, timedelta
import io
import numpy as np
import heapq
import json
import operator
import pandas as pd
from collections import Counter
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stop_words import get_stop_words
import nltk
from nltk.stem.porter import PorterStemmer
import numpy.random as nprnd
#nltk.download()
stop_words = get_stop_words('english')

from gensim.models.doc2vec import LabeledSentence, Doc2Vec

import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

/Users/christophelanternier/anaconda/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
path_to_data = '../data/'

##########################
# load files #                           
##########################

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
#training_info = pd.read_csv(path_to_data+"training_info2.csv",sep=',', header=0, index_col=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)
#test_info = pd.read_csv(path_to_data+"test_info2.csv",sep=',', header=0, index_col=0)

In [3]:
# Correct dates and put datetime format
# We do that because we noticed test_set is only composed of email posterior to the ones of train_set. 
# Datetime format allows to simulate posteriority in our train/test split
from datetime import datetime

for row in training_info.sort(['date']).iterrows():
    date = row[1]['date']
    if date[:3] == '000':
        date = '2' + date[1:]
        
    training_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

for row in test_info.sort(['date']).iterrows():
    date = row[1]['date']
        
    test_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

/Users/christophelanternier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/christophelanternier/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [38]:
# Get the sender column in training_info and test_info
# !! très long, a faire tourner plus tard et enregistrer les resultats dans un CSV
def get_sender(query_mid, training):
    for row in training.iterrows():
        mids = row[1]['mids'].split()
        for mid in mids:
            if int(mid) == query_mid:
                sender = row[1]['sender']
                break
    return sender

test_info['sender'] = 0
for row in test_info.iterrows():
    if row[0]%100==0:
        print row[0]
    query_mid = row[1]['mid']
    test_info.loc[row[0], 'sender'] = get_sender(query_mid, test)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


## Text Embeddings

#### Doc2vec

function "most_similar" from doc2vec doesn't work. Impossible to get closest documents. Maybe the corpus isn't big enough to train a neural net. Trying to recover closest docs with cosine similarity doesn't seem to work either.

In [45]:
# In this cell we prepare labels for emails in case we want to perform a multilabel classification

#To see the code that allowed to create that list, see "archive"
with io.open('../data/person_id.txt') as json_data:
    person_id = json.load(json_data)

#Create labels for emails
labels = []
for row in training_info.iterrows():
    recipients_id = []
    for recipients in row[1]['recipients'].split():
        if '@' in recipients:
            #print recipients
            recipients_id.append(person_id[recipients])
            
    labels.append(recipients_id)
documents = training_info['body'].values

from sklearn.preprocessing import MultiLabelBinarizer
labels_binary = MultiLabelBinarizer().fit_transform(labels)

In [110]:
#Create a list of documents that fit Doc2Vec expected format. For some reason document have to receive a 'label', 
#but they are more like tags, don't know if it could be used for classification purposes. 
documents = []

for row in training_info.iterrows():
    document = LabeledSentence(words=row[1]['body'].split(), tags=['SENTENCE_'+str(row[0])]) 
    documents.append(document)

In [114]:
model = Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=5, hs=0, min_count=2, workers=cores)
model.build_vocab(documents)

In [120]:
for epoch in range(10):
    random.shuffle(documents)
    model.train(documents)
    print 'done for epoch: ', str(epoch) 

done for epoch:  0
done for epoch:  1
done for epoch:  2
done for epoch:  3
done for epoch:  4
done for epoch:  5
done for epoch:  6
done for epoch:  7
done for epoch:  8
done for epoch:  9


In [ ]:
# save model
model.save('../data/enron.d2v')

### TF-IDF

## First Algorithm

The idea here is to embed the email body, find its sender, find the 30 closest emails in the embedding space, which were written by the sender, construct a dictionnary of recipients those 30 emails were addressed to, and pick the 10 most frequent reciepient

In [13]:
stemmer = PorterStemmer()

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

def most_similar_sklearn(array_embedding_sparse, mail_tfidf):
    
    similarities = cosine_similarity(array_embedding_sparse, mail_tfidf)
    if int(round(sorted(similarities[:,0], reverse=True)[0])) ==1:
        closest_ids = similarities[:,0].argsort()[::-1][1:]
    else:
        closest_ids = similarities[:,0].argsort()[::-1]
    
    return closest_ids

def get_sender(query_mid, training):
    for row in training.iterrows():
        mids = row[1]['mids'].split()
        for mid in mids:
            if int(mid) == query_mid:
                sender = row[1]['sender']
                break
    return sender

def get_n_closest_emails(sender, n, closest_ids, training, training_info, mail_date):
    # Get all emails' mids from query sender
    all_emails_from_sender_mids = [int(k) for k in training[training['sender']==sender]['mids'].values[0].split()]

    # Get emails' index from query sender
    training_info_anterior = training_info[training_info['date'] <= mail_date]
    all_emails_from_sender_ids = training_info_anterior[training_info_anterior['mid'].isin(all_emails_from_sender_mids)].index.values

    # Get the closest emails WRITTEN BY THE SENDER
    closest_ids_per_sender = []
    for idx in closest_ids:
        if idx in all_emails_from_sender_ids:
            closest_ids_per_sender.append(idx)
        if len(closest_ids_per_sender) == n:
            break
            
    return closest_ids_per_sender

def get_10_recipients(closest_ids_per_sender, training_info, closest_emails_dates):
    dic_of_recipients = {}
    weight = len(closest_ids_per_sender)+1
    for idx in closest_ids_per_sender:
        recipients = training_info.loc[idx,'recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                if recipient not in dic_of_recipients.keys():
                    dic_of_recipients[recipient] = 0.5*weight*closest_emails_dates['weight_date'][idx]
                else:
                    dic_of_recipients[recipient] += 0.5*weight*closest_emails_dates['weight_date'][idx]
        weight-=1

    suggested_10_recipients = heapq.nlargest(10, dic_of_recipients, key=dic_of_recipients.get)
    
    return suggested_10_recipients

def mean_ap(suggested_10_recipients, ground_truth):
    MAP = 0
    correct_guess = 0
    for i, suggestion in enumerate(suggested_10_recipients):
        if suggestion in ground_truth:
            correct_guess +=1
            MAP += float(correct_guess)/(i+1)
    MAP = float(MAP)/min(10, len(ground_truth))
    return MAP

## Test the algorithm

In [189]:
X_train_info.index = range(X_train_info.shape[0])
duplicate_test_set_ids = []
count=0
for mail in X_test_info['body'].values:
    count+=1
    if count%100==0:
        print count
    mail_tfidf = tfidf.transform([mail])
    similarities = cosine_similarity(array_embedding_sparse, mail_tfidf)
    
    closest_id = similarities[:,0].argsort()[::-1][0]
    duplicate_test_set_ids.append(closest_id)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [47]:
submission = True
training_info = training_info.sort_values(by='date')

if submission:
    # submission procedure
    X_train_info = training_info
    X_test_info = test_info
    
else:
    # test procedure
    split_date=datetime(2001, 8, 25)
    X_train_info = training_info[training_info.date <= split_date]
    #X_test_info = training_info[training_info.date > split_date]
    
    #Randomize selection of test set:
    X_test_info = training_info[training_info.date > split_date]
    mask = nprnd.choice(range(X_test_info.shape[0]), size=1000, replace=False)
    X_test_info.index = range(X_test_info.shape[0])
    X_test_info = X_test_info[X_test_info.index.isin(mask)]

    X_train_info = training_info

In [48]:
if submission:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(np.concatenate((X_train_info['body'].values,X_test_info['body'].values)))
    array_embedding_sparse = array_embedding_sparse[:X_train_info.shape[0]]
else:
    #With porter stemming:
    #tfidf = TfidfVectorizer(tokenizer= tokenize, stop_words = stop_words)
    #Without stemming:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(X_train_info['body'].values)

In [49]:
all_mean_ap = []
all_ground_truth = []
all_suggestions = []
results = pd.DataFrame(columns=['recipients'])
results.index.name = 'mid'
# number of closest neighbors to collect recipients from:
n = 30

#re-arrange train index
X_train_info.index = range(X_train_info.shape[0])

t1 = datetime.now()

count = 1
query_id = 45
for query_id in X_test_info.index.values:

    count+=1
    if count%100==0:
        print count

    mail = X_test_info['body'][query_id]
    mail_date = X_test_info['date'][query_id]
    query_mid = X_test_info['mid'][query_id]

    mail_tfidf = tfidf.transform([mail])
    closest_ids = most_similar_sklearn(array_embedding_sparse, mail_tfidf)

    # find the sender from the query email
    # For the real prediction, replace training by test
    if submission:
        sender = get_sender(query_mid, test)
        #sender = X_test_info[X_test_info.mid == query_mid]['sender'].values[0]
        
    else:
        sender = get_sender(query_mid, training)
        #sender = X_train_info[X_train_info.mid == query_mid]['sender'].values[0]

    # find the closest emails (written by the sender) to the query one
    closest_ids_per_sender = get_n_closest_emails(sender, n, closest_ids, training, X_train_info, mail_date)

    closest_emails_dates = pd.DataFrame(X_train_info['date'][closest_ids_per_sender].sort_values())
    closest_emails_dates['weight_date'] = range(1, len(closest_ids_per_sender)+1)

    if closest_emails_dates[closest_emails_dates['date']>mail_date].shape[0] > 0:
        print query_id

    # Create dictionnary of all recipient for the 30 most similar emails, and get frequency
    # For the moment it is only the brute frequency, maybe we could refine this by adding wheights according to the closseness of the email
    suggested_10_recipients = get_10_recipients(closest_ids_per_sender, X_train_info, closest_emails_dates)

    if submission:
        string_recipients = ''
        for k in suggested_10_recipients:
            string_recipients+=k + ' '

        results.loc[query_mid, 'recipients'] = string_recipients
    else:
        ground_truth = X_test_info['recipients'][query_id].split()
        all_suggestions.append(suggested_10_recipients)
        ground_truth.append(ground_truth)
        all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))
    #print ground_truth
    #print
    #print suggested_10_recipients
print datetime.now()-t1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
0:04:42.486672


In [46]:
np.mean(all_mean_ap)

0.30680133393172082

In [50]:
results.to_csv('../submission/submission_30_with_weights_0-5_linear_dates_linear.csv')

## Baseline

In [158]:
################################
# create some handy structures #                    
################################
                            
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids

# save all unique sender names
all_senders = emails_ids_per_sender.keys()

In [159]:
# create address book with frequency information for each user
address_books = {}
i = 0

for sender, ids in emails_ids_per_sender.iteritems():
    recs_temp = []
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        recs_temp.append(recipients)
    # flatten    
    recs_temp = [elt for sublist in recs_temp for elt in sublist]
    # compute recipient counts
    rec_occ = dict(Counter(recs_temp))
    # order by frequency
    sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    # save
    address_books[sender] = sorted_rec_occ
    
    if i % 10 == 0:
        print i
    i += 1
    

0
10
20
30
40
50
60
70
80
90
100
110
120


In [160]:
#save all unique recipient names    
all_recs = list(set([elt[0] for sublist in address_books.values() for elt in sublist]))

# save all unique user names 
all_users = []
all_users.extend(all_senders)
all_users.extend(all_recs)
all_users = list(set(all_users))

In [161]:
#############
# baselines #                           
#############

# will contain email ids, predictions for random baseline, and predictions for frequency baseline
predictions_per_sender = {}

# number of recipients to predict
k = 10

for index, row in test.iterrows():
    name_ids = row.tolist()
    sender = name_ids[0]
    # get IDs of the emails for which recipient prediction is needed
    ids_predict = name_ids[1].split(' ')
    ids_predict = [int(my_id) for my_id in ids_predict]
    random_preds = []
    freq_preds = []
    # select k most frequent recipients for the user
    k_most = [elt[0] for elt in address_books[sender][:k]]
    for id_predict in ids_predict:
        # select k users at random
        random_preds.append(random.sample(all_users, k))
        # for the frequency baseline, the predictions are always the same
        freq_preds.append(k_most)
    predictions_per_sender[sender] = [ids_predict,random_preds,freq_preds]	

In [162]:
#################################################
# write predictions in proper format for Kaggle #                           
#################################################

path_to_results = '../submission/'

with open(path_to_results + 'predictions_random.csv', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        random_preds = preds[1]
        for index, my_preds in enumerate(random_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')

with open(path_to_results + 'predictions_frequency.csv', 'wb') as my_file:
    my_file.write('mid,recipients' + '\n')
    for sender, preds in predictions_per_sender.iteritems():
        ids = preds[0]
        freq_preds = preds[2]
        for index, my_preds in enumerate(freq_preds):
            my_file.write(str(ids[index]) + ',' + ' '.join(my_preds) + '\n')